# 🌍 LULC Random Forest Classification - Interactive Notebook

This notebook provides an interactive way to:
- Explore your data
- Train and evaluate models
- Visualize results
- Run predictions

## 1. Setup and Imports

In [ ]:
import sys
sys.path.append('..')

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import rasterio
from pathlib import Path
import yaml

# Import our custom modules
from scripts.extract_features import FeatureExtractor
from scripts.train_random_forest import LULCTrainer
from scripts.run_rf_inference import LULCPredictor
from scripts.evaluate_random_forest import LULCEvaluator

# Configure plotting
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')
%matplotlib inline

print('✓ Imports successful!')

## 2. Load Configuration

In [ ]:
with open('../config/config.yaml', 'r') as f:
    config = yaml.safe_load(f)

print('Configuration loaded:')
print(f"  Classes: {len(config['classes'])}")
print(f"  Features: {len(config['features']['bands']) + len(config['features']['indices'])}")
print(f"  Model: Random Forest with {config['model']['n_estimators']} trees")

## 3. Explore Training Data

In [ ]:
# Load training data
X_train = np.load('../data/training/features.npy')
y_train = np.load('../data/training/labels.npy')

print(f'Training samples: {X_train.shape[0]:,}')
print(f'Features per sample: {X_train.shape[1]}')
print(f'\nClass distribution:')

for class_id, class_info in config['classes'].items():
    count = np.sum(y_train == class_id)
    percentage = (count / len(y_train)) * 100
    print(f"  {class_info['name']:15s}: {count:8,} ({percentage:5.2f}%)")

In [ ]:
# Visualize class distribution
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

class_names = [config['classes'][i]['name'] for i in sorted(config['classes'].keys())]
class_counts = [np.sum(y_train == i) for i in range(len(class_names))]

# Bar plot
ax1.bar(class_names, class_counts, color='steelblue')
ax1.set_title('Class Distribution', fontsize=14, fontweight='bold')
ax1.set_ylabel('Number of Samples')
ax1.tick_params(axis='x', rotation=45)

# Pie chart
colors = [tuple(np.array(config['classes'][i]['color'])/255) for i in range(len(class_names))]
ax2.pie(class_counts, labels=class_names, autopct='%1.1f%%', colors=colors, startangle=90)
ax2.set_title('Class Proportions', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

## 4. Feature Analysis

In [ ]:
# Feature statistics
feature_names = config['features']['bands'] + config['features']['indices']

fig, axes = plt.subplots(3, 4, figsize=(16, 10))
axes = axes.flatten()

for i, feature_name in enumerate(feature_names):
    if i >= len(axes):
        break
    
    ax = axes[i]
    
    # Plot histogram for each class
    for class_id, class_info in config['classes'].items():
        class_data = X_train[y_train == class_id, i]
        ax.hist(class_data, bins=50, alpha=0.5, label=class_info['name'])
    
    ax.set_title(feature_name, fontweight='bold')
    ax.set_xlabel('Value')
    ax.set_ylabel('Frequency')
    if i == 0:
        ax.legend(fontsize=8)

# Hide unused subplots
for i in range(len(feature_names), len(axes)):
    axes[i].axis('off')

plt.suptitle('Feature Distributions by Class', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

## 5. Train Model

In [ ]:
# Initialize trainer
trainer = LULCTrainer('../config/config.yaml')

# Split data
X_train_split, X_val, y_train_split, y_val = trainer.prepare_data(X_train, y_train, test_size=0.2)

# Train model
model = trainer.train_model(X_train_split, y_train_split)

print('\n✓ Model training complete!')

## 6. Evaluate Model

In [ ]:
# Evaluate on validation set
report, cm = trainer.evaluate_model(X_val, y_val, save_plots=False)

# Plot confusion matrix
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=class_names, yticklabels=class_names)
plt.title('Confusion Matrix', fontsize=16, fontweight='bold')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.tight_layout()
plt.show()

In [ ]:
# Feature importance
importances = model.feature_importances_
indices = np.argsort(importances)[::-1]

plt.figure(figsize=(12, 6))
plt.bar(range(len(importances)), importances[indices])
plt.xticks(range(len(importances)), [feature_names[i] for i in indices], rotation=45, ha='right')
plt.title('Feature Importance', fontsize=16, fontweight='bold')
plt.xlabel('Feature')
plt.ylabel('Importance')
plt.tight_layout()
plt.show()

print('\nTop 5 Most Important Features:')
for i in range(min(5, len(importances))):
    idx = indices[i]
    print(f'  {i+1}. {feature_names[idx]}: {importances[idx]:.4f}')

## 7. Save Model

In [ ]:
# Save the trained model
trainer.save_model('../models/random_forest/model.pkl')
print('✓ Model saved successfully!')

## 8. Run Inference on New Image

In [ ]:
# Initialize predictor
predictor = LULCPredictor('../models/random_forest/model.pkl', '../config/config.yaml')

# Run prediction on a test image
test_image_path = '../data/raw/sample_image_1.tif'
prediction_map = predictor.predict_image(test_image_path)

In [ ]:
# Visualize prediction
fig, axes = plt.subplots(1, 2, figsize=(16, 7))

# Original image (RGB composite)
with rasterio.open(test_image_path) as src:
    r = src.read(3)
    g = src.read(2)
    b = src.read(1)
    rgb = np.dstack([r, g, b])
    # Normalize for display
    rgb = (rgb - rgb.min()) / (rgb.max() - rgb.min())

axes[0].imshow(rgb)
axes[0].set_title('Original Image (RGB)', fontsize=14, fontweight='bold')
axes[0].axis('off')

# Classified map
cmap = plt.cm.colors.ListedColormap([tuple(np.array(config['classes'][i]['color'])/255) 
                                      for i in range(len(config['classes']))])
im = axes[1].imshow(prediction_map, cmap=cmap, vmin=0, vmax=4)
axes[1].set_title('LULC Classification', fontsize=14, fontweight='bold')
axes[1].axis('off')

# Add colorbar
cbar = plt.colorbar(im, ax=axes[1], ticks=range(5))
cbar.set_ticklabels(class_names)

plt.tight_layout()
plt.show()

## 9. Calculate Statistics

In [ ]:
# Calculate area statistics
pixel_size = 30  # meters (assuming Landsat)
pixel_area = (pixel_size ** 2) / 1e6  # km²

print('LULC Area Statistics:')
print('=' * 60)

total_area = 0
for class_id, class_info in config['classes'].items():
    count = np.sum(prediction_map == class_id)
    area = count * pixel_area
    percentage = (count / prediction_map.size) * 100
    total_area += area
    
    print(f"{class_info['name']:15s}: {area:8.2f} km² ({percentage:5.2f}%)")

print('=' * 60)
print(f"{'Total':15s}: {total_area:8.2f} km²")

## 10. Export Results

In [ ]:
# Save classification to GeoTIFF
output_path = '../results/notebook_classification.tif'
predictor._save_prediction(prediction_map, output_path, None)

print(f'✓ Classification saved to {output_path}')

## Summary

This notebook demonstrated:
1. ✅ Loading and exploring training data
2. ✅ Analyzing feature distributions
3. ✅ Training a Random Forest model
4. ✅ Evaluating model performance
5. ✅ Running inference on new imagery
6. ✅ Visualizing results
7. ✅ Calculating area statistics

Next steps:
- Try different hyperparameters
- Add more training data
- Experiment with feature engineering
- Test on different geographic regions